In [1]:
import pandas as pd
from dataclasses import dataclass
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from pathlib import Path
from scipy.stats import pearsonr
from matplotlib import gridspec

/Users/philippinedecaix/anaconda3/envs/iblenv/lib/python3.10/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


In [9]:
@dataclass
class Params: 
    """
    Parameters for PSTH and photometry analysis.

    Attributes
    ----------
    pre_time : float
        Time before event to include in the analysis window (in seconds).
    post_time : float
        Time after event to include in the analysis window (in seconds).
    bin_size : float
        Size of time bins for the histogram (in seconds).
    baseline_time : float
        Time window before the event used for baseline fluorescence calculation (in seconds).
    """
    pre_time: float=4
    post_time: float=6
    bin_size: float=0.1
    baseline_time: float=2

In [10]:
class Photometry: 
    """
    Class for analyzing photometry data aligned to behavioral events.

    Parameters
    ----------
    eid : str
        Experiment ID used to load data files.

    trial_num : list, optional
        Indices of trial numbers to select from the session. 

    """

    def __init__(self, eid, trial_num=None):
        
        # private instances
        self.__pathfile = '/Users/philippinedecaix/IBL/neuromodulators/extracted_photometry_data/DA/'
        self.__eid = eid
        # now __convert_trial_num always returns a boolean mask
        self.__trial_num = self.__convert_trial_num(trial_num)
        
        # load & subset
        self.__trials_df = self.load_trials()[self.__trial_num]
        self.__gcamp_df  = self.load_photometry()
       
        # self.__trial_num = self.__convert_trial_num(trial_num)
        
        # if trial_num!=None:
        #     self.__trials_df = self.load_trials()[self.__trial_num]
        # else: 
        #     self.__trials_df = self.load_trials()
            
        # self.__gcamp_df = self.load_photometry()

        # public instances
        self.firstMovement_times = self.__trials_df['firstMovement_times']
        self.contrastRight = self.__trials_df['contrastRight']
        self.choice = self.__trials_df['choice']
        self.contrastLeft = self.__trials_df['contrastLeft']
        self.response_times = self.__trials_df['response_times']
        self.feedbackType = self.__trials_df['feedbackType']
        self.probabilityLeft = self.__trials_df['probabilityLeft']
        self.stimOn_times = self.__trials_df['stimOn_times']
        self.goCue_times = self.__trials_df['goCue_times']
        self.feedback_times = self.__trials_df['feedback_times']
    
    def __convert_trial_num(self, trial_num): 
        # total_trial = self.load_trials().shape[0]
        # if len(trial_num) == 2:
        #     trial_num = [True if i in range(trial_num[0], trial_num[1]) else False for i in range(total_trial)]
        # return trial_num

        # total number of trials in the session
        n = self.load_trials().shape[0]
        
        # 1) if user passed nothing, select all trials
        if trial_num is None:
            return np.ones(n, dtype=bool)
        
        # 2) if they passed a two‐element [start, stop], make a range mask
        if isinstance(trial_num, (list, tuple)) and len(trial_num) == 2:
            start, stop = trial_num
            return np.array([start <= i < stop for i in range(n)], dtype=bool)
        
        # 3) if they passed a list of indices, mark those True
        if isinstance(trial_num, (list, np.ndarray)):
            mask = np.zeros(n, dtype=bool)
            mask[trial_num] = True
            return mask
        
        # 4) otherwise assume it’s already a boolean mask
        return trial_num

    # functions for loading
    def load_photometry(self):
        df = pd.read_parquet(self.__pathfile + self.__eid + '_gcamp.pqt')
        return df

    def load_trials(self): 
        df = pd.read_parquet(self.__pathfile + self.__eid + '_trials.pqt')
        return df

    def load_isosbestic(self): 
        df = pd.read_parquet(self.__pathfile + self.__eid + '_isosbestic.pqt')
        return df
    
        
    def __bin_photometry(self, align_times: np.ndarray, params: Params):
        """
        Align and bin the photometry signal around each event time,
        compute ΔF/F (%) per bin, and return (n_trials × n_bins) array
        plus the bin-center time scale.
        """
        signal = self.__gcamp_df.values[:, 0]
        times = self.__gcamp_df.index.values
    
        # number of bins before and after event
        n_pre = int(np.ceil(params.pre_time / params.bin_size))
        n_post = int(np.ceil(params.post_time / params.bin_size))
        # full tscale including edges
        full_t = np.arange(-n_pre, n_post + 1) * params.bin_size
    
        # we'll drop the last edge to get centers
        tscale = 0.5 * (full_t[:-1] + full_t[1:])
        n_bins = tscale.size
    
        bins = np.full((align_times.size, n_bins), np.nan)
    
        for i, t0 in enumerate(align_times):
            # find window in the raw trace
            start_idx = np.searchsorted(times, t0 - params.pre_time)
            end_idx   = np.searchsorted(times, t0 + params.post_time)
    
            # baseline window [t0 - baseline_time, t0)
            b0, b1 = np.searchsorted(times, [t0 - params.baseline_time, t0])
            F0 = np.nanmean(signal[b0:b1]) if (b1 > b0) else np.nan
    
            if np.isnan(F0) or F0 == 0:
                # skip this trial if baseline invalid
                continue
    
            seg_times = times[start_idx:end_idx] - (t0 - params.pre_time)
            seg_sig   = signal[start_idx:end_idx]
    
            # compute bin indices
            xind = np.floor(seg_times / params.bin_size).astype(int)
    
            # keep only bins in [0, n_bins)
            valid = (xind >= 0) & (xind < n_bins)
            if not np.any(valid):
                # no data in window => leave row as NaN
                continue
    
            x_valid = xind[valid]
            sig_valid = seg_sig[valid]
    
            # sum & count per bin
            counts = np.bincount(x_valid, minlength=n_bins)
            sums   = np.bincount(x_valid, weights=sig_valid, minlength=n_bins)
    
            with np.errstate(divide='ignore', invalid='ignore'):
                mean_per_bin = sums / counts
                mean_per_bin[counts == 0] = np.nan
    
            # ΔF/F (%) per bin
            df_f = (mean_per_bin - F0) / F0 * 100
            bins[i, :] = df_f
    
        return bins, tscale

    
    
    def get_psth(self, events, params: Params, side=None, contrast=None, feedback_type=None, bias=None):
        """
        Compute PSTH aligned to a specific event with optional filtering by the side, contrast level, 
        feedback type, and bias in experimental setup. 
        
        (PSTH) Peri-Stimulus Time Histogram. It’s a way to visualize how your fluorescence signal 

        Params
        -------
        events : str
            Column name of the event times to align to.
        params : Params
            Analysis parameters.
        side : str, optional
            'left' or 'right' trials.
        contrast : list, optional
            List of contrast levels to include.
        feedback_type : float, optional
            correct trials (1.0) or incorrect trials (-1.0).
        bias: int, optional
            50-50 trials (0) or 20-80 trials (1). 

        Returns
        -------
        psth_avg : np.ndarray of shape (n_bins,)
            The average binned dF/F (%) signal across all selected trials.
            
        tscale : np.ndarray of shape (n_bins,)
            The time points (in seconds) corresponding to the center of each bin.
            
        psth_contrast : dict of {float: np.ndarray of shape (n_bins,)}
            Dictionary mapping each contrast level to the average binned dF/F (%) signal 
            across all trials for that specific contrast level.
        """

        events = np.array(self.__trials_df[events])  # e.g. 'firstMovement_times'
        dF_F, tscale = self.__bin_photometry(events, params)  
       
        # print(f'dF_F shape: {dF_F.shape}')
        # print(f'tscale shape: {tscale.shape}')
        # print(f'trials_df shape {self.__trials_df.shape}')
        # print(f'gcamp_df shape {self.__gcamp_df.shape}')

        left_idx = ~np.isnan(self.contrastLeft)
        self.__left_psth = np.nanmean(dF_F[left_idx], axis=0)
        self.__left_psth_sem = stats.sem(dF_F[left_idx], axis=0, nan_policy='omit')
        right_idx = ~np.isnan(self.contrastRight)
        self.__right_psth = np.nanmean(dF_F[right_idx], axis=0)
        self.__right_psth_sem = stats.sem(dF_F[right_idx], axis=0, nan_policy='omit')
        
        correct_idx = self.feedbackType == 1.0
        self.__correct_psth = np.nanmean(dF_F[correct_idx], axis=0)
        self.__correct_psth_sem = stats.sem(dF_F[correct_idx], axis=0, nan_policy='omit')
        incorrect_idx = self.feedbackType == -1.0
        self.__incorrect_psth = np.nanmean(dF_F[incorrect_idx], axis=0)
        self.__incorrect_psth_sem = stats.sem(dF_F[incorrect_idx], axis=0, nan_policy='omit')

        no_bias_idx = self.probabilityLeft == 0.5
        self.__no_bias_psth = np.nanmean(dF_F[no_bias_idx], axis=0)
        self.__no_bias_psth_sem = stats.sem(dF_F[no_bias_idx], axis=0, nan_policy='omit')
        bias_idx = (self.probabilityLeft == 0.2) | (self.probabilityLeft == 0.8)
        self.__bias_psth = np.nanmean(dF_F[bias_idx], axis=0)
        self.__bias_psth_sem = stats.sem(dF_F[bias_idx], axis=0, nan_policy='omit')

        if side=='left':
            trials = dF_F[left_idx]
            trials_data = self.__trials_df[left_idx]

        elif side=='right':
            trials = dF_F[right_idx]
            trials_data = self.__trials_df[right_idx]
       
        else: 
            # all trials
            trials = dF_F
            trials_data = self.__trials_df
        
        if feedback_type == 1.0:
            trials = dF_F[trials_data['feedbackType'] == 1.0]
            trials_data = trials_data[trials_data['feedbackType'] == 1.0]
            
        elif feedback_type == -1.0:
            trials = dF_F[trials_data['feedbackType'] == -1.0]
            trials_data = trials_data[trials_data['feedbackType'] == -1.0]
        
        elif feedback_type!=None: 
            raise Exception("feedback type must be a float -1.0 (incorrect) or 1.0 (correct). ")
 
        if bias == 0:
            trials = dF_F[trials_data['probabilityLeft'] == 0.5]
            trials_data = trials_data[trials_data['probabilityLeft'] == 0.5]

        elif bias == 1:
            trials = dF_F[(trials_data['probabilityLeft'] == 0.8) | (trials_data['probabilityLeft'] == 0.2)]
            trials_data = trials_data[(trials_data['probabilityLeft'] == 0.8) | (trials_data['probabilityLeft'] == 0.2)]
        
        elif bias!=None: 
            raise Exception("bias must be an integer value 0 (50-50 no bias) or 1 (80-20 bias).")
        
        # take the average over all identified trials
        psth_avg = np.nanmean(trials, axis=0) 
        # self._psth_sem = np.nanstd(trials, axis=0) / np.sqrt(trials.shape[0]) 
        self._psth_sem = stats.sem(trials, axis=0, nan_policy='omit')
        
        # store psth for different contrast levels
        psth_contrast = {}
        self._psth_contrast_sem = {}
        
        if contrast!=None:
            contrast.sort()
            for level in contrast:
                # trial indices for this difficulty
                idx = np.where((trials_data['contrastLeft'] == level)|(trials_data['contrastRight'] == level))[0] 
                if len(idx) == 0:
                    continue  # skip if no trials of this level
        
                psth_contrast[level] = np.nanmean(trials[idx], axis=0) # store for this level
                # self._psth_contrast_sem[level] = np.nanstd(trials[idx], axis=0) / np.sqrt(trials[idx].shape[0])
                self._psth_contrast_sem[level] = stats.sem(trials[idx], axis=0, nan_policy='omit')
            
            return psth_avg, tscale, psth_contrast
        
        else:
            return psth_avg, tscale
    
    def sessions_avg_psth(eids, events, params):
        
        bin_store = []
        for eid in eids: 
            try:
                bin = Photometry(eid=eid).get_psth(events, params)[0]
                bin_store.append(bin)
            except:
                continue
        
        return np.mean(bin_store, axis=0)
    
    def plot_psth(self, events, params: Params):
        """
        Plot PSTH for all trials.
        """

        if not hasattr(self, '_psth_sem'):
            raise ValueError("Call get_psth() before plotting to compute SEM.")

        psth, tscale = self.get_psth(events, params)
        
        plt.figure(figsize=(8, 6))
        sem = self._psth_sem
        plt.plot(tscale, psth, c='g')
        plt.fill_between(tscale, psth - sem, psth + sem, color='g', alpha=0.2)
        plt.xlabel('Time (s)')
        plt.ylabel('ΔF/F(%)')
        # plt.ylim(-0.5, 0.5)
        plt.title(f'PSTH of Dopamine Signal relative to {events}')
        plt.axvline(0, c='k', linestyle='--')
        plt.show()
    
    def sessions_plot(eids, events, params):

        bin_store = []

        for eid in eids: 
            try:
                bin, tscale = Photometry(eid=eid).get_psth(events, params)
                bin_store.append(bin)
            except:
                continue

        bin_avg = np.mean(bin_store, axis=0)
        bin_sem = np.nanstd(bin_store, axis=0) / np.sqrt(len(bin_store))

        plt.figure(figsize=(8, 6))
        sem = bin_sem
        plt.plot(tscale, bin_avg, c='g')
        plt.fill_between(tscale, bin_avg - sem, bin_avg + sem, color='g', alpha=0.2)
        plt.xlabel('Time (s)')
        plt.ylabel('ΔF/F(%)')
        plt.ylim(-1, 1.2)
        plt.title(f'PSTH of fluorescence relative to {events}')
        plt.axvline(0, c='k', linestyle='--')
        plt.show()

        # sns.kdeplot(bin_avg, bw_adjust=0.1)

    def plot_psth_by_contrast_level(self, events, contrast, params: Params):
        """
        Plot PSTH by side or contrast levels.
        """
        
        if not hasattr(self, '_psth_sem'):
            raise ValueError("Call get_psth() before plotting to compute SEM.")

        tscale, psth_contrast = self.get_psth(events, params, contrast=contrast)[1:]
        
        plt.figure(figsize=(8, 6))
        n_lines = len(contrast)
        palette = sns.color_palette("YlOrBr", n_lines) 
        idx = 0

        for level, psth in psth_contrast.items():
            sem = self._psth_contrast_sem[level]
            plt.plot(tscale, psth, color=palette[idx], label=f'Contrast level: {level}')
            plt.fill_between(tscale, psth - sem, psth + sem,
                        color=palette[idx], alpha=0.2)
            idx += 1

        plt.xlabel('Time (s)')
        plt.ylabel('ΔF/F(%)')
        plt.ylim(-1, 1.2)
        plt.title(f'PSTH of Dopamine Signaling Relative to {events} by Trial Difficulty')
        plt.axvline(0, c='k', linestyle='--')
        plt.legend()
        plt.show()
    
    def plot_psth_by_feedback_type(self, events, params: Params):
        
        tscale = self.get_psth(events, params)[1]

        plt.figure(figsize=(8, 6))
        n_lines = 2
        palette = sns.color_palette("YlOrBr", n_lines) 

        plt.plot(tscale, self.__correct_psth, color=palette[0], label='correct trials')
        plt.plot(tscale, self.__incorrect_psth, color=palette[1], label='incorrect trials')
        plt.fill_between(tscale, self.__correct_psth - self.__correct_psth_sem, self.__correct_psth + self.__correct_psth_sem,
                        color=palette[0], alpha=0.2)
        plt.fill_between(tscale, self.__incorrect_psth - self.__incorrect_psth_sem, self.__incorrect_psth + self.__incorrect_psth_sem,
                        color=palette[1], alpha=0.2)
        plt.xlabel('Time (s)')
        plt.ylabel('ΔF/F(%)')
        plt.title(f'PSTH of Dopamine Signaling Relative to {events} by feedback type')
        plt.axvline(0, c='k', linestyle='--')
        plt.legend()
        plt.show()
    
    def plot_psth_by_bias(self, events, params: Params):
        
        tscale = self.get_psth(events, params)[1]

        plt.figure(figsize=(8, 6))
        n_lines = 2
        palette = sns.color_palette("YlOrBr", n_lines) 

        plt.plot(tscale, self.__no_bias_psth, color=palette[0], label='50-50 trials')
        plt.plot(tscale, self.__bias_psth, color=palette[1], label='80-20 trials')
        plt.fill_between(tscale, self.__no_bias_psth - self.__no_bias_psth_sem, self.__no_bias_psth + self.__no_bias_psth_sem,
                        color=palette[0], alpha=0.2)
        plt.fill_between(tscale, self.__bias_psth - self.__bias_psth_sem, self.__bias_psth + self.__bias_psth_sem,
                        color=palette[1], alpha=0.2)
        plt.xlabel('Time (s)')
        plt.xlabel('Time (s)')
        plt.ylabel('ΔF/F(%)')
        plt.title(f'PSTH of Dopamine Signaling Relative to {events} by bias probability')
        plt.axvline(0, c='k', linestyle='--')
        plt.legend()
        plt.show()
    
    def plot_psth_by_side(self, events, params: Params):
        
        tscale = self.get_psth(events, params)[1]

        plt.figure(figsize=(8, 6))
        n_lines = 2
        palette = sns.color_palette("YlOrBr", n_lines) 

        plt.plot(tscale, self.__left_psth, color=palette[0], label='left trials')
        plt.plot(tscale, self.__right_psth, color=palette[1], label='right trials')
        plt.fill_between(tscale, self.__left_psth - self.__left_psth_sem, self.__left_psth + self.__left_psth_sem,
                        color=palette[0], alpha=0.2)
        plt.fill_between(tscale, self.__right_psth - self.__right_psth_sem, self.__right_psth + self.__right_psth_sem,
                        color=palette[1], alpha=0.2)
        plt.xlabel('Time (s)')
        plt.ylabel('ΔF/F(%)')
        plt.title(f'PSTH of Dopamine Signaling Relative to {events} by side')
        plt.axvline(0, c='k', linestyle='--')
        plt.legend()
        plt.show()

In [11]:
def extract_dopamine_0_25s(behavior_df: pd.DataFrame, params: Params) -> pd.DataFrame:
    """
    Loop through each eid, bin photometry around stimOn_times,
    average ΔF/F over 0–1s window, return per-trial DataFrame.
    """
    rows = []
    for eid, subdf in behavior_df.groupby('eid', sort=False):
        try:
            P = Photometry(eid=eid)
            align = P._Photometry__trials_df['stimOn_times'].values
            bins, tscale = P._Photometry__bin_photometry(align, params)

            # mask for 0 ≤ t < 1s
            mask = (tscale >= 0) & (tscale < 0.250)
            # guard against empty mask
            if not np.any(mask):
                per_trial = np.full(bins.shape[0], np.nan)
            else:
                per_trial = np.nanmean(bins[:, mask], axis=1)

            for avg, (_, beh) in zip(per_trial, subdf.iterrows()):
                rows.append({
                    'eid':           beh['eid'],
                    'subject':       beh['subject'],
                    'session_num':   beh['session_num'],
                    'trial_num':     beh['trial_num'],
                    'dopamine_0_1s': avg
                })

        except (FileNotFoundError, IOError):
            for _, beh in subdf.iterrows():
                rows.append({
                    'eid':           beh['eid'],
                    'subject':       beh['subject'],
                    'session_num':   beh['session_num'],
                    'trial_num':     beh['trial_num'],
                    'dopamine_0_1s': np.nan
                })

    return pd.DataFrame(rows)

In [12]:
def extract_slope_dopamine_0_25s(behavior_df: pd.DataFrame, params: Params) -> pd.DataFrame:
    """
    Loop through each eid, bin photometry around stimOn_times,
    compute the slope of ΔF/F over 0–0.25s window, return per-trial DataFrame.
    """
    rows = []
    for eid, subdf in behavior_df.groupby('eid', sort=False):
        try:
            P = Photometry(eid=eid)
            align = P._Photometry__trials_df['stimOn_times'].values
            bins, tscale = P._Photometry__bin_photometry(align, params)

            # mask for 0 ≤ t < 0.25s
            mask = (tscale >= 0) & (tscale < 0.250)

            n_trials = bins.shape[0]
            per_trial = np.full(n_trials, np.nan)

            if np.any(mask):
                times = tscale[mask]
                for i in range(n_trials):
                    y = bins[i, mask]
                    valid = ~np.isnan(y)
                    # need at least two points to fit a line
                    if np.sum(valid) > 1:
                        slope, _ = np.polyfit(times[valid], y[valid], 1)
                        per_trial[i] = slope
                    # else leave as NaN

            for slope, (_, beh) in zip(per_trial, subdf.iterrows()):
                rows.append({
                    'eid':             beh['eid'],
                    'subject':         beh['subject'],
                    'session_num':     beh['session_num'],
                    'trial_num':       beh['trial_num'],
                    'dopamine_slope':  slope
                })

        except (FileNotFoundError, IOError):
            for _, beh in subdf.iterrows():
                rows.append({
                    'eid':             beh['eid'],
                    'subject':         beh['subject'],
                    'session_num':     beh['session_num'],
                    'trial_num':       beh['trial_num'],
                    'dopamine_slope':  np.nan
                })

    return pd.DataFrame(rows)

In [14]:
def compute_session_means(df):
    """
    Computes the mean of 'right', 'left', and 'bias' grouped by
    ['eid', 'session_num', 'subject'].

    Parameters:
        df (pd.DataFrame): Input DataFrame with at least the columns:
            ['eid', 'session_num', 'subject', 'right', 'left', 'bias']

    Returns:
        pd.DataFrame: A DataFrame with columns:
            ['eid', 'session_num', 'subject', 'mean_right', 'mean_left', 'mean_bias']
    """
    grouped = df.groupby(['eid', 'session_num', 'subject']).agg(
        mean_right=('right', 'mean'),
        mean_left=('left', 'mean'),
        mean_bias=('bias', 'mean')
    ).reset_index()

    return grouped

In [15]:
def extract_mean_slope_dopamine_learning(behavior_df: pd.DataFrame, params: Params) -> pd.DataFrame:
    """
    For each eid, bin photometry around stimOn_times and compute the mean slope
    of ΔF/F over 0–0.25s window separately for:
      - trials where stimulus is on the right (contrast >= 0.5)
      - trials where stimulus is on the left (contrast <= -0.5)

    Returns one row per session with:
    ['eid', 'subject', 'session_num', 'stimulusrightCR', 'stimulusleftCL']
    """
    rows = []
    for eid, subdf in behavior_df.groupby('eid', sort=False):
        try:
            P = Photometry(eid=eid)
            align = P._Photometry__trials_df['stimOn_times'].values
            bins, tscale = P._Photometry__bin_photometry(align, params)

            mask = (tscale >= 0) & (tscale < 0.250)
            times = tscale[mask]

            stim_right_trials = subdf['contrast'] >= 0.5
            stim_left_trials = subdf['contrast'] <= -0.5

            def compute_slopes(indices):
                slopes = []
                for i in indices:
                    y = bins[i, mask]
                    valid = ~np.isnan(y)
                    if np.sum(valid) > 1:
                        slope, _ = np.polyfit(times[valid], y[valid], 1)
                        slopes.append(slope)
                return np.nanmean(slopes) if slopes else np.nan

            # Get indices for right and left trials
            right_idx = subdf[stim_right_trials].reset_index(drop=True).index.tolist()
            left_idx = subdf[stim_left_trials].reset_index(drop=True).index.tolist()


            slope_right = compute_slopes(right_idx)
            slope_left = compute_slopes(left_idx)

            first_row = subdf.iloc[0]
            rows.append({
                'eid': first_row['eid'],
                'subject': first_row['subject'],
                'session_num': first_row['session_num'],
                'stimulusrightCR': slope_right,
                'stimulusleftCL': slope_left
            })

        except (FileNotFoundError, IOError):
            first_row = subdf.iloc[0]
            rows.append({
                'eid': first_row['eid'],
                'subject': first_row['subject'],
                'session_num': first_row['session_num'],
                'stimulusrightCR': np.nan,
                'stimulusleftCL': np.nan
            })

    return pd.DataFrame(rows)

In [16]:
from scipy.stats import pearsonr
import pandas as pd

def correlate_predictors_with_photometry_slopes(session_df):
    """
    For each subject, compute Pearson correlations between dopamine slope
    (stimulusleftCL / stimulusleftCR / stimulusrightCR) and predictor weights
    (mean_left / mean_right) across sessions.

    Returns:
        A DataFrame with:
        ['subject', 'comparison', 'n_sessions', 'pearson_r', 'p_value']
    """

    correlations = []

    comparisons = [
        ('stimulusrightCR', 'mean_right'),
        ('stimulusrightCR', 'mean_left'),
        ('stimulusleftCL', 'mean_right'),
        ('stimulusleftCL', 'mean_left')
    ]

    for subject, group in session_df.groupby('subject'):
        for slope_var, predictor_var in comparisons:
            df = group[[slope_var, predictor_var]].dropna()
            n = len(df)
            if n > 1:
                r, p = pearsonr(df[slope_var], df[predictor_var])
            else:
                r, p = float('nan'), float('nan')
            
            correlations.append({
                'subject': subject,
                'comparison': f"{slope_var} vs {predictor_var}",
                'n_sessions': n,
                'pearson_r': r,
                'p_value': p
            })

    return pd.DataFrame(correlations)


In [17]:
# usage
params = Params(pre_time=4, post_time=6, bin_size=0.1, baseline_time=2)
dopamine_df = extract_slope_dopamine_0_25s(see, params)

In [18]:
dopamine_df

,eid,subject,session_num,trial_num,dopamine_slope
0,606af20c-40c0-4702-850c-ba0ddfe99523,ZFM-03447,3,1.0,NaN
1,606af20c-40c0-4702-850c-ba0ddfe99523,ZFM-03447,3,2.0,NaN
2,606af20c-40c0-4702-850c-ba0ddfe99523,ZFM-03447,3,3.0,NaN
3,606af20c-40c0-4702-850c-ba0ddfe99523,ZFM-03447,3,4.0,NaN
4,606af20c-40c0-4702-850c-ba0ddfe99523,ZFM-03447,3,5.0,NaN
...,...,...,...,...,...
186175,821d049f-7f6b-4724-a3f7-72d5ea30556c,ZFM-04026,66,327.0,NaN
186176,821d049f-7f6b-4724-a3f7-72d5ea30556c,ZFM-04026,66,328.0,NaN
186177,821d049f-7f6b-4724-a3f7-72d5ea30556c,ZFM-04026,66,329.0,NaN
186178,821d049f-7f6b-4724-a3f7-72d5ea30556c,ZFM-04026,66,330.0,NaN


In [19]:
non_nan =  dopamine_df[ dopamine_df['dopamine_slope'].notna() ]

In [20]:
yay=(pd.merge(see,dopamine_df, on=['eid','trial_num', 'session_num', 'subject'], how='inner'))

In [21]:
yay_non_na=yay[ yay['dopamine_slope'].notna() ]

In [22]:
high=pd.read_parquet('performance_dat.parquet')
high_perf=high[['subject', 'session_num', 'proportion_correct']]
high_perf

,subject,session_num,proportion_correct
0,ZFM-03447,1,1.000000
1,ZFM-03447,2,1.000000
2,ZFM-03447,3,0.352941
3,ZFM-03447,4,0.444934
4,ZFM-03447,5,0.397490
...,...,...,...
419,ZFM-04026,62,0.945055
420,ZFM-04026,63,0.957746
421,ZFM-04026,64,0.732558
422,ZFM-04026,65,0.611940


In [23]:
al_df=(pd.merge(yay, high_perf, on=['subject', 'session_num'], how='left'))

In [27]:
all_df_non_nan_high_pL= al_df[(al_df['dopamine_slope'].notna()) &
                            (al_df['probabilityLeft'] == 0.5) &
                    (al_df['session_num']> 30)]
all_df_non_nan_high_pL

,stimOnTrigger_times,goCueTrigger_times,quiescencePeriod,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,feedback_times,...,genotype,session_num,stage,contrast,trial_num,bias,right,left,dopamine_slope,proportion_correct
10508,8.821200,8.874300,0.613898,8.903900,9.426800,-1.0,8.874200,0.0000,0.1250,9.426900,...,DAT,31,training,0.1250,2.0,1.368078,-0.593995,-7.335716,-3.282663,0.750000
10509,13.057400,13.107300,0.532449,13.137400,13.537700,1.0,13.107200,-0.5000,0.0000,13.537800,...,DAT,31,training,-0.5000,3.0,1.371249,-0.593994,-7.402424,-14.560000,0.750000
10510,16.205600,16.258200,0.539792,16.286800,16.688600,1.0,16.258100,-0.5000,0.0000,16.688700,...,DAT,31,training,-0.5000,4.0,1.374910,-0.593993,-7.364973,13.732847,0.750000
10511,19.880200,19.924700,0.603286,19.952300,23.883700,-1.0,19.924600,-0.1250,0.0000,23.913800,...,DAT,31,training,-0.1250,5.0,1.379070,-0.593993,-7.221217,-19.656390,0.750000
10513,30.644400,30.691000,0.478266,30.720700,31.098600,1.0,30.690900,-1.0000,0.0000,31.098700,...,DAT,31,training,-1.0000,7.0,1.381983,-0.593992,-7.158712,-0.264504,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180041,418.256700,418.318900,0.677001,418.346900,418.650800,1.0,418.318800,-0.0625,0.0000,418.650900,...,DAT,55,bias,-0.0625,86.0,1.017821,2.887227,-6.655494,-4.222110,0.956522
180042,421.438299,421.501699,0.541909,421.532299,421.935199,-1.0,421.501599,0.0000,0.0625,421.935299,...,DAT,55,bias,0.0625,87.0,1.074295,2.887098,-6.655541,5.269018,0.956522
180043,426.061800,426.134900,0.575615,426.161900,431.389500,-1.0,426.134800,0.0000,0.1250,431.389600,...,DAT,55,bias,0.1250,88.0,1.119862,2.886965,-6.655587,2.459523,0.956522
180044,434.326300,434.385200,0.677076,434.416000,434.690200,1.0,434.385100,-0.0625,0.0000,434.690300,...,DAT,55,bias,-0.0625,89.0,1.156321,2.886825,-6.655634,2.741637,0.956522


In [37]:
# get left 
# Sort by subject and start_time
all_df_non_nan_high_pL = all_df_non_nan_high_pL.sort_values(['subject', 'right']).copy()

# Initialize trial_bin as float to allow NaN and future int assignment
all_df_non_nan_high_pL['trial_bin'] = np.nan

# Loop per subject
for subject, group in all_df_non_nan_high_pL.groupby('subject'):
    n = len(group)
    if n >= 4:
        bins = pd.qcut(range(n), q=4, labels=[1, 2, 3, 4])
        # Convert to int explicitly before assigning
        bins = bins.astype(int)
        all_df_non_nan_high_pL.loc[group.index, 'trial_bin'] = bins
        
all_df_non_nan_high_pL_r=all_df_non_nan_high_pL.copy()
# Convert to nullable integer type if desired
all_df_non_nan_high_pL_r['trial_bin'] = all_df_non_nan_high_pL['trial_bin'].astype('Int64')


In [35]:
# get right
# Sort by subject and start_time
all_df_non_nan_high_pL = all_df_non_nan_high_pL.sort_values(['subject', 'left']).copy()

# Initialize trial_bin as float to allow NaN and future int assignment
all_df_non_nan_high_pL['trial_bin'] = np.nan

# Loop per subject
for subject, group in all_df_non_nan_high_pL.groupby('subject'):
    n = len(group)
    if n >= 4:
        bins = pd.qcut(range(n), q=4, labels=[1, 2, 3, 4])
        # Convert to int explicitly before assigning
        bins = bins.astype(int)
        all_df_non_nan_high_pL.loc[group.index, 'trial_bin'] = bins
        
all_df_non_nan_high_pL_l=all_df_non_nan_high_pL.copy()
# Convert to nullable integer type if desired
all_df_non_nan_high_pL_l['trial_bin'] = all_df_non_nan_high_pL['trial_bin'].astype('Int64')


In [38]:
all_df_non_nan_high_pL_r

,stimOnTrigger_times,goCueTrigger_times,quiescencePeriod,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,feedback_times,...,session_num,stage,contrast,trial_num,bias,right,left,dopamine_slope,proportion_correct,trial_bin
17894,196.225100,196.291700,0.520245,196.322700,256.322800,0.0,196.291600,0.0000,1.0000,256.351400,...,53,bias,1.0000,12.0,3.523984,-1.932701,-4.468427,0.102908,0.633333,1
17893,173.417100,173.475900,0.511564,173.505700,192.424500,-1.0,173.475800,0.0000,0.0000,192.453200,...,53,bias,0.0000,11.0,3.760574,-1.932686,-4.349709,0.901280,0.633333,1
17892,157.152000,157.209600,0.467775,157.237700,169.625800,-1.0,157.209500,-0.1250,0.0000,169.656300,...,53,bias,-0.1250,10.0,3.990222,-1.932671,-4.230991,0.041159,0.633333,1
17891,138.265500,138.326700,0.504188,138.357200,153.353100,-1.0,138.326600,-1.0000,0.0000,153.384200,...,53,bias,-1.0000,9.0,4.210581,-1.932656,-4.113180,-0.061756,0.633333,1
17895,260.019700,260.070800,0.446159,260.100900,260.525800,1.0,259.640600,-1.0000,0.0000,260.525900,...,53,bias,-1.0000,13.0,3.541003,-1.932649,-4.587146,0.401319,0.633333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173471,444.341300,444.390700,0.546931,444.418100,447.075600,-1.0,444.390600,0.0000,0.0625,447.075700,...,44,training,0.0625,94.0,-0.824606,9.386013,-16.593297,0.082121,0.975610,4
173467,424.279800,424.324600,0.489935,424.351700,424.854400,1.0,424.324500,-0.0625,0.0000,424.854500,...,44,training,-0.0625,90.0,-0.882111,9.387872,-16.593312,0.061548,0.975610,4
173470,438.302500,438.356700,0.452244,438.384400,438.669000,1.0,438.356600,-1.0000,0.0000,438.669100,...,44,training,-1.0000,93.0,-0.844354,9.387967,-16.593300,-1.375219,0.975610,4
173469,434.467000,434.523300,0.521646,434.550900,435.571000,1.0,434.523200,0.0000,0.0000,435.571100,...,44,training,0.0000,92.0,-0.864103,9.389922,-16.593304,0.277233,0.975610,4


In [41]:
#taking all the contrast
r_df_s=all_df_non_nan_high_pL_r[(all_df_non_nan_high_pL_r['contrast']>0) &(all_df_non_nan_high_pL_r['trial_bin']==1)]
r_df_e=all_df_non_nan_high_pL_r[(all_df_non_nan_high_pL_r['contrast']>0) &(all_df_non_nan_high_pL_r['trial_bin']==4)]
l_df_s=all_df_non_nan_high_pL_l[(all_df_non_nan_high_pL_l['contrast']<0) &(all_df_non_nan_high_pL_l['trial_bin']==1)]
l_df_e=all_df_non_nan_high_pL_l[(all_df_non_nan_high_pL_l['contrast']>0) &(all_df_non_nan_high_pL_l['trial_bin']==4)]
r_df_04019= r_df[(r_df['subject']=='ZFM-04019')&(r_df['session_num']==18)]

In [42]:
#r_df=r_df[r_df['trial_bin'].notna()]
r_df

,stimOnTrigger_times,goCueTrigger_times,quiescencePeriod,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,feedback_times,...,session_num,stage,contrast,trial_num,bias,right,left,dopamine_slope,proportion_correct,trial_bin
17894,196.225100,196.291700,0.520245,196.322700,256.322800,0.0,196.291600,0.0,1.0000,256.351400,...,53,bias,1.0000,12.0,3.523984,-1.932701,-4.468427,0.102908,0.633333,1
17890,129.749400,129.807100,0.433767,129.835100,133.952000,-1.0,129.807000,0.0,1.0000,133.952100,...,53,bias,1.0000,8.0,4.285737,-1.932641,-4.108789,0.462794,0.633333,1
17889,109.915800,109.977100,0.407590,110.007600,127.049000,-1.0,109.977000,0.0,1.0000,127.049100,...,53,bias,1.0000,7.0,4.334390,-1.932633,-4.104398,1.193382,0.633333,1
17887,66.889400,66.957900,0.457780,66.987100,83.771300,-1.0,66.957800,0.0,0.0625,83.771400,...,53,bias,0.0625,5.0,4.377138,-1.932631,-4.095616,0.308340,0.633333,1
17886,47.622700,47.694600,0.638297,47.727600,64.044000,-1.0,47.694500,0.0,0.2500,64.044100,...,53,bias,0.2500,4.0,4.392316,-1.932629,-4.091225,-0.133733,0.633333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173463,406.726300,406.774100,0.503393,406.805500,407.387600,-1.0,406.774000,0.0,0.1250,407.387700,...,44,training,0.1250,86.0,-0.897402,9.371774,-16.593311,-0.246110,0.975610,4
173472,451.608199,451.657199,0.623923,451.687999,452.704099,-1.0,451.657099,0.0,1.0000,452.704199,...,44,training,1.0000,95.0,-0.834347,9.379566,-16.593293,0.564731,0.975610,4
173466,421.034500,421.091200,0.457205,421.122300,421.632700,-1.0,421.091100,0.0,1.0000,421.632800,...,44,training,1.0000,89.0,-0.889257,9.383867,-16.593313,1.077663,0.975610,4
173471,444.341300,444.390700,0.546931,444.418100,447.075600,-1.0,444.390600,0.0,0.0625,447.075700,...,44,training,0.0625,94.0,-0.824606,9.386013,-16.593297,0.082121,0.975610,4


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

def compute_bias_stats(df, contrast_side_col='right', dopamine_col='dopamine_slope'):
    """
    Compute Pearson correlation between contrast side and dopamine slope for each subject/session.

    Parameters:
    - df (pd.DataFrame): The input dataframe containing at least 'subject', 'session_num',
                         contrast side, and dopamine slope columns.
    - contrast_side_col (str): Name of the column representing the contrast side (e.g., 'right').
    - dopamine_col (str): Name of the column representing dopamine slope.

    Returns:
    - pd.DataFrame: A dataframe with correlation stats per subject and session.
    """
    results = []

    for (subj, session), grp in df.groupby(['subject', 'session_num'], sort=True):
        x = grp[contrast_side_col].dropna()
        y = grp[dopamine_col].dropna()
        valid = x.index.intersection(y.index)
        x, y = x.loc[valid], y.loc[valid]

        if len(x) >= 3:
            r, p = pearsonr(x, y)
        else:
            r, p = np.nan, np.nan

        results.append({
            'subject':     subj,
            'session_num': session,
            'n_trials':    len(x),
            'pearson_r':   r,
            'p_value':     p
        })

    return pd.DataFrame(results)

In [50]:
import matplotlib.colors as mc
import colorsys

def lighten_color(color, amount=0.5):
    """
    Lighten the given color by blending it with white.
    amount=0 → original color, amount=1 → pure white.
    """
    try:
        c = mc.cnames[color]
    except:
        c = color
    r, g, b = mc.to_rgb(c)
    # blend each channel with white (1.0)
    r_l = r + (1.0 - r) * amount
    g_l = g + (1.0 - g) * amount
    b_l = b + (1.0 - b) * amount
    return (r_l, g_l, b_l)


In [ ]:
def plot_sessionwise_correlations(session_stats):
    """
    Plots session-wise Pearson correlations by subject with significance indicated.

    Parameters:
    - session_stats (pd.DataFrame): Output from `compute_bias_stats()` containing columns:
      'subject', 'session_num', 'pearson_r', and 'p_value'.
    """
    # Assign a unique base color per subject
    subjects = session_stats['subject'].unique()
    palette  = sns.color_palette("tab10", len(subjects))
    colmap   = dict(zip(subjects, palette))

    fig = plt.figure(figsize=(8, 6))
    gs  = gridspec.GridSpec(1, 2, width_ratios=[4, 1], wspace=0.05)

    # Left panel: scatter plot
    ax = fig.add_subplot(gs[0])
    for subj, grp in session_stats.groupby('subject'):
        base    = colmap[subj]
        light   = lighten_color(base, amount=0.7)
        sig_mask = grp['p_value'] < 0.05

        ax.scatter(
            grp.loc[~sig_mask, 'session_num'],
            grp.loc[~sig_mask, 'pearson_r'],
            color=light,
            s=60,
            edgecolors='none',
            label=subj
        )
        ax.scatter(
            grp.loc[sig_mask, 'session_num'],
            grp.loc[sig_mask, 'pearson_r'],
            color=base,
            s=60,
            edgecolors='k',
            linewidths=0.5
        )

    ax.axhline(0, color='gray', linestyle='--', lw=0.7)
    ax.set_xlabel('Session Number')
    ax.set_ylabel('Pearson r')
    ax.set_title('Session‐wise Correlation')
    ax.legend(
        title='Subject',
        loc='upper right',
        bbox_to_anchor=(0.98, 0.98),
        frameon=False
    )

    # Right panel: histogram of Pearson r values
    axh = fig.add_subplot(gs[1], sharey=ax)
    sig = session_stats['p_value'] < 0.05
    ns  = ~sig
    axh.hist(
        session_stats.loc[ns, 'pearson_r'],
        bins=20,
        orientation='horizontal',
        color='lightgray',
        alpha=0.6,
        label='p ≥ 0.05'
    )
    axh.hist(
        session_stats.loc[sig, 'pearson_r'],
        bins=20,
        orientation='horizontal',
        color='black',
        alpha=0.8,
        label='p < 0.05'
    )
    axh.set_xlabel('Count')
    axh.tick_params(axis='y', left=False, labelleft=False)
    axh.legend(loc='upper right')

    plt.tight_layout()
    plt.show()